# El proceso de búsqueda de elementos similares

En este tutorial vamos a repasar todo el proceso de búsqueda de elementos similares, con el objetivo de buscar, de un conjunto de 10 textos de poesías de Pablo Neruda, cuales son los dos más similares. 

Destacamos las siguientes etapas.
- Shingling. Esto es, transformar el texto a conjuntos. Ya lo hemos demostrado en otro tutorial, pero dejamos el código por continuidad. 
- Minhashing. Para poder calcular la similitud, computaremos una firma de Minhash de tamaño 5 para cada uno de los conjuntos, en este caso, cada uno de los shingles de los textos. 
- Comparación de firmas de Minhash para estimar la similitud de Jaccard. Cuando tenemos la firma de minhash para cada texto, nos concentramos en comparar sus firmas de MinHash. Entonces, compararemos con la similitud de Jaccard solo aquellos textos que presenten la misma firma de MinHash. 

## Shingling

La técnica de *k*-shingling consiste en transformar un texto (es decir, un string) a un conjunto formado por todos los substring de tamaño *k* de ese texto, incluyendo espacios y otros caracteres no lexicográficos. 

Veamos como hacer esto para un conjunto de 10 poesías escritas por Pablo Neruda, parte de su obra de Odas Elementales. 

In [4]:
### Para almacenar los distintos textos creamos un diccionario. 
odas = {}

### Vamos a subir cada texto indexado por un keyword distinto en este diccionario. 
### Para hacerlos más legibles, reemplazamos los fin de línea por un espacio. 

with open("oda_alegria.txt", "r", encoding='utf-8') as file:
    text = file.read().replace('\n', ' ')
odas['alegria']=text

with open("oda_caldillo.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['caldillo']=text

with open("oda_feliz.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['feliz']=text

with open("oda_libro.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['libro']=text

with open("oda_mar.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['mar']=text

with open("oda_poetas.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['poetas']=text

with open("oda_tiempo.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['tiempo']=text

with open("oda_tristeza.txt", "r", encoding='utf-8') as file:
    text = file.read().replace('\n', ' ')
odas['tristeza']=text


with open("oda_valparaiso.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['valparaiso']=text


with open("oda_vino.txt", "r") as file:
    text = file.read().replace('\n', ' ')
odas['vino']=text

### Como vemos, el resultado es que cada uno de estos textos es un string, 
### indexado en el diccionario por un keyword diferente. 

print(odas)

{'alegria': 'Alegría hoja verde caída en la ventana, minúscula claridad recién nacida, elefante sonoro, deslumbrante moneda, a veces ráfaga quebradiza, pero más bien pan permanente, esperanza cumplida, deber desarrollado. Te desdeñé, alegría. Fui mal aconsejado. La luna me llevó por sus caminos. Los antiguos poetas me prestaron anteojos y junto a cada cosa un nimbo oscuro puse, sobre la flor una corona negra, sobre la boca amada un triste beso. Aún es temprano. Déjame arrepentirme. Pensé que solamente si quemaba mi corazón la zarza del tormento, si mojaba la lluvia mi vestido en la comarca cárdena del luto, si cerraba los ojos a la rosa y tocaba la herida, si compartía todos los dolores, yo ayudaba a los hombres. No fui justo. Equivoqué mis pasos y hoy te llamo, alegría.  Como la tierra eres necesaria.  Como el fuego sustentas los hogares.  Como el pan eres pura.  Como el agua de un río eres sonora.  Como una abeja repartes miel volando.  Alegría, fui un joven taciturno, hallé tu cabel

In [5]:
### El siguiente pedazo de código genera un diccionario, indexado bajo los mismos keywords que los textos, 
### En este diccionario almacenamos el resultado de hacer k-shingling en el texto. 

### Probemos con k = 3

odas_shingles = {}
k = 3

### iteramos sobre todas las odas
for (name,text) in odas.items(): 
    
    ### Es importante declarar los shingles como un set (conjunto), de forma que no hayan duplicados
    odas_shingles[name] = set()
    
    ### y nos concentramos en todos los substrings entre las posiciones i y la i+k+1, para un 
    ### i que parte desde el inicio del texto 
    
    for i in range(len(text) - k-1):
        shingle = text[i:i+k]
        odas_shingles[name].add(shingle)
        

In [6]:
### Este es el resultado de hacer shingling al primer texto, la Oda a la Alegría. 

print(odas_shingles['alegria'])

{'aba', 'ere', ' má', 'den', 'epa', 'asc', 're ', ' ba', 'man', 'rro', 'eri', 'or ', 'rtí', 'no ', 'unt', 'ás ', 'za ', 'a q', 'a d', 'jos', 'zar', 'sto', 'ia.', 'n l', 'ntr', 'lo ', 'a t', 'nec', 'so.', 'e d', 'pen', 'ten', ' pi', ' La', ' ho', 'n t', 'ert', 'ugu', 'go ', 'a u', 'mar', 'rza', 'un ', 'cib', 'tre', 'ter', 'ier', ', f', 'str', 'ent', 'ho ', ' li', 'is ', 'e b', 'ues', 'gus', 'spe', 'o q', 'eta', 'la,', 'bie', 'tod', 'sca', ' au', 'ena', 'n b', 'lé ', 'áfa', 'bre', 'rme', 'ibr', 'Com', ' pe', 'dre', 'eja', 'mab', 'pus', 'ech', 'on ', ' ro', 'oli', 'cua', 'd r', 'Los', 'to!', 'orm', 'muc', 'o! ', 'na ', 'o a', 'pe ', 'pob', 'río', 'tos', 'ria', 'uto', 'ris', 'ndo', 'oet', 'r c', 'Con', 'Voy', ' bo', 'ía.', 's h', 'to.', 'luc', 'cad', 'Fui', 'ad ', 'e t', 'l m', ' cl', 'n e', 'a v', 'erm', 'No ', 'uiv', 'lie', '. A', 'e. ', 'nor', 'r e', 'err', 'uel', 'sar', ', h', 'te,', 'rad', 'tie', 'lo,', 'ien', 'ios', 'cam', 'e: ', '. L', 'ué ', 'no.', ' A ', 'sé ', 'r d', 'a f', 'ún '

## Jaccard Similarity

Vamos a comparar cual de estas 10 odas es la más parecida entre sí, según la similitud de Jaccard. 

In [7]:
### Lo primero es una funcion que calcula la similitud de Jaccard entre dos conjuntos. 
### Recordemos que se define como la proporcion entre el tamaño de la intersección de los conjuntos, 
### dividido por el tamaño de su unión. 

def jaccard_similarity(set1, set2):
    # Computa la similitud de Jaccard entre dos sets
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

In [8]:
### Un par de pruebas: 
### - La similitud de un conjunto con si mismo debe ser 1
### - La similitud de un conjunto con el vacio debe ser cero
### - Probamos cuanto es la similitud entre los primeros dos textos. 

print(jaccard_similarity(odas_shingles['alegria'],odas_shingles['alegria']))
print(jaccard_similarity(odas_shingles['alegria'],{}))
print(jaccard_similarity(odas_shingles['alegria'],odas_shingles['caldillo']))

1.0
0.0
0.29086115992970124


Ahora vamos a calcular la similitud para todas las $10 \choose 2$ combinaciones. 
El resultado irá a una matriz de 10x10, donde la entrada $[i][j]$ es el valor 
de la similitud entre la i-ésima oda y la j-ésima oda. 

In [9]:
### Armamos la matriz 

n = len(odas_shingles)
similarity_matrix = [[0 for i in range(n)] for j in range(n)]

### con esto podemos usar names[i] para conseguir el nombre de la i-esima oda. 
names = list(odas_shingles.keys())

### Llenamos la matriz
for i in range(n):
    for j in range(n):
        if i != j:
            similarity = jaccard_similarity(odas_shingles[names[i]], odas_shingles[names[j]])
            similarity_matrix[i][j] = similarity
            
for row in similarity_matrix:
    print(row)


[0, 0.29086115992970124, 0.24478178368121442, 0.3175787728026534, 0.3258584442887176, 0.3288062902072909, 0.2981691368788143, 0.2358036573628489, 0.3349019607843137, 0.338474025974026]
[0.29086115992970124, 0, 0.25966183574879226, 0.28780487804878047, 0.31098546042003233, 0.3140495867768595, 0.3034188034188034, 0.23479318734793186, 0.3243992606284658, 0.3346228239845261]
[0.24478178368121442, 0.25966183574879226, 0, 0.23987206823027718, 0.2736750651607298, 0.25591586327782645, 0.28381642512077293, 0.22922636103151864, 0.24366471734892786, 0.25435897435897437]
[0.3175787728026534, 0.28780487804878047, 0.23987206823027718, 0, 0.324468085106383, 0.33489461358313816, 0.3137254901960784, 0.2555309734513274, 0.3399482312338223, 0.3157894736842105]
[0.3258584442887176, 0.31098546042003233, 0.2736750651607298, 0.324468085106383, 0, 0.3656716417910448, 0.30812101910828027, 0.2332761578044597, 0.3789004457652303, 0.3638426626323752]
[0.3288062902072909, 0.3140495867768595, 0.25591586327782645, 0

En este caso lo podemos hacer de forma sencilla, ya que son solo 10 textos pequeños. Cuando la cantidad de conjuntos es muy grande, probar para todas las combinaciones es muy costoso, y debemos usar MinHashing. 

Para terminar, veamos cuales dos odas son las mas similares. 

In [10]:
maximo = 0
for i in range(n):
    for j in range(n):
        valor = similarity_matrix[i][j]
        if maximo < valor: 
            i_max = i
            j_max = j
            maximo = valor
print(i_max,j_max,maximo)

4 8 0.3789004457652303


In [11]:
names[4],names[8]

('mar', 'valparaiso')

Las mas similares son las odas al mar y a Valparaíso. 
Esto tiene sentido, ya que Valparaíso es una ciudad a orillas del mar, y por tanto a ratos se hablarán cosas parecidas. 

## Minhashing

Para calcular las firmas de minhash, vamos a usar cuatro funciones de hash, las que intuitivamente jugarán el rol de una permutación de la matriz de elementos. 
Veamos primero que funciones de hash podemos usar. Estas derivan de usar la operación módulo (%), y tienen la ventaja de que simulan permutaciones de la matriz de elementos - conjuntos: $h(i)$ nos dice en que orden está la fila $i$ en la permutación. 

In [19]:
### Contamos la cantidad de elementos totales que tenemos, 
### como el tamaño de la union de todos los conjuntos de shingles. 

un = set()
for (name,shingle) in odas_shingles.items():
        un = un.union(shingle)
        
print(len(un))

2387


La matriz de elementos tiene, entonces, 2349 filas. Las permutaciones serán funciones que transformen números entre 
0 y 2348 a numeros entre 0 y 2348. Definimos 5 de estas funciones, aunque podría ser cualquier función de hash. 

In [ ]:
def h0(n):
    return (n+4) % 2387
def h1(n):
    return (2*n+1) % 2387
def h2(n):
    return (2*n) % 2387
def h3(n):
    return (3*n+300) % 2387
def h4(n):
    return (n+1000) % 2387


In [14]:
### Veamos algunos ejemplos. En estas funciones, la antigua fila 1 sería la siguiente fila en la permutación: 

print(h0(1),h1(1),h2(1),h3(1),h4(1))

5 3 2 303 1001


In [15]:
### Y la antigua fila 100 sería la siguiente fila en la permutación: 

print(h0(100),h1(100),h2(100),h3(100),h4(100))

104 201 200 600 1100


### Calculando la firma de minhash de las odas

Como ya sabemos, la matriz de elementos - conjuntos tiene 2349 filas, y 10 columnas, una por cada oda. Vamos a reducir esto a una matriz de 5 filas y 10 columnas: para cada columna, representando una oda, entregamos la firma de minhash correspondiente a la función de hash hi.

In [16]:
### Creamos la matriz FH con las filas y columnas correspondientes. 
### Inicialmente la matriz tiene solo valores 3000, cualquier valor mas grande 
### que la cantidad de elementos sirve. 

n = len(odas_shingles)
FH = [[3000 for j in range(n)] for i in range(5)]

### Tomamos la union de todos los elementos en el conjunto llamado un, 
### y para iterar sobre estos, lo transformamos a una lista ordenada

for (name,shingle) in odas_shingles.items():
    un = un.union(shingle)
    sorted_union = sorted(un)

### con esto podemos usar names[i] para conseguir el nombre de la i-esima oda. 
names = list(odas_shingles.keys())

             ### iteramos cada elemento
for e in range(0,len(sorted_union)): 
    ### iteramos cada columna, correspondiente a la cantidad de textos (odas en nuestro caso)  
    for i in range(0,n):
        ### si el elemento e no está en el texto i, no hacemos nada
        if sorted_union[e] not in odas_shingles[names[i]]:
            pass
        ### si el elemento si está, computamos los valores de h(e) y 
        else:
            FH[0][i]=min(FH[0][i],h0(e))
            FH[1][i]=min(FH[1][i],h1(e))
            FH[2][i]=min(FH[2][i],h2(e))
            FH[3][i]=min(FH[3][i],h3(e))
            FH[4][i]=min(FH[4][i],h4(e)) 

In [17]:
### Ahora imprimimos la matriz
for row in FH:
    print(row)  

[4, 3, 0, 6, 0, 0, 0, 12, 0, 0]
[1, 8, 7, 5, 1, 1, 9, 17, 5, 0]
[0, 7, 6, 4, 0, 0, 8, 16, 4, 0]
[0, 0, 0, 0, 3, 0, 0, 6, 0, 0]
[0, 3, 0, 0, 0, 0, 0, 8, 7, 0]


Ahora podemos pensar en que cada oda está representada por su firma de MinHash, que es 
la columna correspondiente, y podemos usar esa firma para revisar si acaso dos textos son similares. Así, si dos textos tienen la misma firma de minhash, entonces es probable que sean muy parecidos. 


Por ejemplo, estas son las columnas de las odas 5 y 9, correspondientes a los índices 4 y 8 de la matriz, que identificamos como similares antes: 

In [18]:
print(' 4     8 ')
print('---------')  
for row in FH:
    print('|'+str(row[4])+'|   |'+str(row[8])+'|')  

 4     8 
---------
|0|   |0|
|1|   |5|
|0|   |4|
|3|   |0|
|0|   |7|


Si bien hay algunas coincidencias, la firma no es igual. Esto es consistente con la similitud de Jaccard que vimos: si bien es la mas alta, no es un valor muy cercano a 1, si no que está cerca de 0.4. Esto coincide con la proporcion de hashes que son iguales en nuestro caso.  